Experiments with [regularized evolution](https://arxiv.org/pdf/1802.01548.pdf)

In [46]:
import random
import collections
import copy
import scipy
from scipy.optimize import dual_annealing

In [47]:
DIM = 10

In [13]:
class Model(object):
  def __init__(self):
    self.params = []
    self.accuracy = None

In [48]:
# Rosenbrock in 10D
def evalModel(params):
  sum = 0.0
  for i in range(DIM-1):
    sum = sum + (100.0*(params[i+1] - (params[i])**2)**2 + (1 - params[i])**2)
  return sum

In [35]:
def randomParameters():
  params = []
  for _ in range(DIM):
    params.append(random.uniform(a=-10.0, b=10.0))
  return params

def mutateParameters(params):
  new_params = copy.deepcopy(params)
  idx = random.randint(0, DIM-1)
  new_params[idx] = random.uniform(a=-10.0, b=10.0)
  return new_params

def mutateParameters_v2(params, mu0=0.7):
  r = random.uniform(a=0.0, b=1.0)
  if r <= mu0:
    new_params = copy.deepcopy(params)
    idx = random.randint(0, DIM-1)
    new_params[idx] = random.uniform(a=-10.0, b=10.0)
    return new_params
  else:
    new_params = []
    for _ in range(DIM):
      new_params.append(random.uniform(a=-10.0, b=10.0))
    return new_params

In [36]:
def regularizedEvolution(cycles, population_size, sample_size):
  population = collections.deque()
  history = []

  # initialize the population
  while (len(population) < population_size):
    model = Model()
    model.params = randomParameters()
    model.accuracy = evalModel(model.params)
    population.append(model)
    history.append(model)

  while (len(history) < cycles):
    # Sample models for the tournament
    sample = []
    while (len(sample) < sample_size):
      candidate = random.choice(list(population))
      sample.append(candidate)

    # Get the best model from sample
    parent = min(sample, key = lambda i: i.accuracy)

    # Add a mutated model and remove the oldest
    child = Model()
    child.params = mutateParameters(parent.params)
    child.accuracy = evalModel(child.params)
    population.append(child)
    history.append(child)
    population.popleft()

  return history

Experiments.

In [43]:
history = regularizedEvolution(cycles=100000, population_size=300, sample_size=50)

In [44]:
best = min(history, key=lambda i: i.accuracy)
print(best.accuracy)
for p in best.params:
  print(p)

11.701168934957808
0.9833844319870373
0.9631215341753965
1.0247622721789895
1.0391251470043983
1.0073030008229793
0.9705574304926774
1.1161136710430437
1.4149288434285818
1.8294680057466746
3.399115610424321


For comparison, let us use dual annealing from scipy. 

In [49]:
lb = [-10.0]*DIM
ub = [10.0]*DIM
res = dual_annealing(evalModel, bounds = list(zip(lb, ub)), seed = 123)

In [50]:
print(res.x)
print(res.fun)

[0.99999997 0.99999995 0.99999987 0.99999976 0.99999967 0.99999935
 0.99999875 0.99999755 0.99999501 0.99999003]
3.681947780085446e-11
